In [1]:
import pandas as pd
import os
import sys
# Add the parent directory to the Python path
parent_dir = os.path.abspath(os.path.join('..'))
sys.path.append(parent_dir)
# from travel_time_matrix import travel_time_matrix
from Metric_Calculation.MetricCalculation import MetricCalculation
parent_dir = os.path.abspath(os.path.join('../..'))
sys.path.append(parent_dir)
from data import data


### 1. Load the travel time matrix result generated using r5py and create Metric Calculation object
    

In [ ]:
# Before calculation
# Ensure the file paths are correct
travel_time_matrix_path = "../results/TTM_CT_hospitals_only_before.csv"
num_of_dest_output_path = "../results/TTM_CT_hospitals_before.csv"
thresholds = [30]
n_travel_time_output_path = "../results/first_hospitals_before.csv"

TTM_Metric = MetricCalculation(travel_time_matrix_path) 
TTM_Metric.filter_destinations( output_path = num_of_dest_output_path, threshold = thresholds )  # Filter the destinations to only include those within 30 minutes travel time
TTM_Metric.get_nth_travel_time(n  = 1, output_path= n_travel_time_output_path)

Generated ../results/TTM_CT_hospitals_before_threshold_30.csv
N-th travel time (N=1) saved to ../results/first_hospitals_before.csv


,from_id,to_id,travel_time
0,5350001.00,773891,73.0
1,5350002.00,54248,100.0
2,5350003.00,10757635,17.0
3,5350004.00,8168605,16.0
4,5350005.00,8168605,13.0
...,...,...,...
578,5350378.24,4616608,40.0
579,5350378.25,6362828,47.0
580,5350378.26,6362828,42.0
581,5350378.27,6362828,84.0


In [ ]:
# After calculation
travel_time_matrix_path = "../results/TTM_CT_hospitals_only_after.csv"
num_of_dest_output_path = "../results/TTM_CT_hospitals_after.csv"
thresholds = [30]
n_travel_time_output_path = "../results/first_hospitals_after.csv"

TTM_Metric = MetricCalculation(travel_time_matrix_path) 
TTM_Metric.filter_destinations( output_path = num_of_dest_output_path, threshold = thresholds )  # Filter the destinations to only include those within 30 minutes travel time
TTM_Metric.get_nth_travel_time(n  = 1, output_path= n_travel_time_output_path)

Generated ../results/TTM_CT_hospitals_after_threshold_30.csv
N-th travel time (N=1) saved to ../results/first_hospitals_after.csv


,from_id,to_id,travel_time
0,5350001.00,773891,73.0
1,5350002.00,54248,100.0
2,5350003.00,10757635,17.0
3,5350004.00,8168605,16.0
4,5350005.00,8168605,13.0
...,...,...,...
578,5350378.24,4616608,40.0
579,5350378.25,6362828,47.0
580,5350378.26,6362828,42.0
581,5350378.27,6362828,84.0


### 2. Combine before and after calculation to one table for visualization

In [ ]:
# Number of destinations within 30 minutes before and after

threshold = 30

before_csv_path = f"../results/TTM_CT_hospitals_before_threshold_{threshold}.csv"
after_csv_path = f"../results/TTM_CT_hospitals_after_threshold_{threshold}.csv"

CTUID_source = "../results/CTUIDs.csv"


total_destinations_df = pd.read_csv(CTUID_source)
area_names_df = pd.read_csv("../../../data/visual_data/CTUID-w-Neighborhood.csv")

# Add 'before' totals

# Add Neighbourhood data
total_destinations_df = MetricCalculation.add_column_with_join(
    total_destinations_df,
    area_names_df,
    base_ctuid_col='CTUID',
    join_ctuid_col='CTUID',
    join_column_name='AREA_NAME',
    new_column_name='Neighbourhood'
)
# display(total_destinations_df.head(20))
before_column_names = {'from_id': 'from_id','to_id': 'to_id', 'CTUID':'CTUID',
        'total_column': f'total_destinations_within_{threshold}_mins_before'}

total_destinations_df = MetricCalculation.add_column_with_join(
    total_destinations_df,
    MetricCalculation.calculate_total_destinations(before_csv_path, CTUID_source, before_column_names),
    base_ctuid_col='CTUID',
    join_ctuid_col='CTUID',
    join_column_name=f'total_destinations_within_{threshold}_mins_before',
    new_column_name=f'total_destinations_within_{threshold}_mins_before'
)
# display(total_destinations_df.head(20))
after_column_names = {'from_id': 'from_id','to_id': 'to_id', 'CTUID':'CTUID',
        'total_column': f'total_destinations_within_{threshold}_mins_after'}

# Add 'after' totals
total_destinations_df = MetricCalculation.add_column_with_join(
    total_destinations_df,
    MetricCalculation.calculate_total_destinations(after_csv_path, CTUID_source, after_column_names),
    base_ctuid_col='CTUID',
    join_ctuid_col='CTUID',
    join_column_name=f'total_destinations_within_{threshold}_mins_after',
    new_column_name=f'total_destinations_within_{threshold}_mins_after'
)


# Calculate Difference
total_destinations_df[f'total_destinations_within_{threshold}_mins_diff'] = (
    total_destinations_df[f'total_destinations_within_{threshold}_mins_after'] - 
    total_destinations_df[f'total_destinations_within_{threshold}_mins_before']
)

display(total_destinations_df.head(20))

# Save the final DataFrame to CSV
csv_output_path = f'../results/hospitals_{threshold}_before_after_diff.csv'
total_destinations_df.to_csv(csv_output_path, index=False)


,CTUID,Neighbourhood,total_destinations_within_30_mins_before,total_destinations_within_30_mins_after,total_destinations_within_30_mins_diff
0,5350128.04,South Eglinton-Davisville,3,3,0
1,5350363.06,Woburn North,2,2,0
2,5350363.07,Woburn North,2,2,0
3,5350378.23,Agincourt North,0,0,0
4,5350378.24,Agincourt North,0,0,0
5,5350010.01,West Queen West,4,4,0
6,5350010.02,West Queen West,7,7,0
7,5350001.00,South Riverdale,0,0,0
8,5350002.00,St Lawrence-East Bayfront-The Islands,0,0,0
9,5350003.00,South Parkdale,2,2,0


In [14]:
# Travel time to first hospital before and after

before_csv_path = f"../results/first_hospitals_before.csv"
after_csv_path = f"../results/first_hospitals_after.csv"

CTUID_source = "../results/CTUIDs.csv"


total_destinations_df = pd.read_csv(CTUID_source)
area_names_df = pd.read_csv("../../../data/visual_data/CTUID-w-Neighborhood.csv")

before_csv = pd.read_csv(before_csv_path)
after_csv = pd.read_csv(after_csv_path)

# Add 'before' totals

# Add Neighbourhood data
total_destinations_df = MetricCalculation.add_column_with_join(
    total_destinations_df,
    area_names_df,
    base_ctuid_col='CTUID',
    join_ctuid_col='CTUID',
    join_column_name='AREA_NAME',
    new_column_name='Neighbourhood'
)
# # display(total_destinations_df.head(20))
# before_column_names = {'from_id': 'from_id','to_id': 'to_id', 'CTUID':'CTUID',
#         'total_column': f'travel_time_to_first_dest_before'}

total_destinations_df = MetricCalculation.add_column_with_join(
    total_destinations_df,
    before_csv,
    base_ctuid_col='CTUID',
    join_ctuid_col='from_id',
    join_column_name=f'travel_time',
    new_column_name=f'travel_time_to_first_dest_before'
)
# display(total_destinations_df.head(20))
# after_column_names = {'from_id': 'from_id','to_id': 'to_id', 'CTUID':'CTUID',
#         'total_column': f'travel_time_to_first_dest_after'}

# Add 'after' totals
total_destinations_df = MetricCalculation.add_column_with_join(
    total_destinations_df,
    after_csv,
    base_ctuid_col='CTUID',
    join_ctuid_col='from_id',
    join_column_name=f'travel_time',
    new_column_name=f'travel_time_to_first_dest_after'
)


# Calculate Difference
total_destinations_df[f'travel_time_to_first_dest_reduction'] = (
    abs(total_destinations_df[f'travel_time_to_first_dest_after'] - 
    total_destinations_df[f'travel_time_to_first_dest_before'])
)

display(total_destinations_df.head(20))

# Save the final DataFrame to CSV
csv_output_path = f'../results/first_hospitals_before_after_reduction.csv'
total_destinations_df.to_csv(csv_output_path, index=False)


,CTUID,Neighbourhood,travel_time_to_first_dest_before,travel_time_to_first_dest_after,travel_time_to_first_dest_reduction
0,5350128.04,South Eglinton-Davisville,22.0,22.0,0.0
1,5350363.06,Woburn North,27.0,27.0,0.0
2,5350363.07,Woburn North,24.0,24.0,0.0
3,5350378.23,Agincourt North,36.0,36.0,0.0
4,5350378.24,Agincourt North,40.0,40.0,0.0
5,5350010.01,West Queen West,8.0,8.0,0.0
6,5350010.02,West Queen West,16.0,16.0,0.0
7,5350001.00,South Riverdale,73.0,73.0,0.0
8,5350002.00,St Lawrence-East Bayfront-The Islands,100.0,100.0,0.0
9,5350003.00,South Parkdale,17.0,17.0,0.0


Pivot the result columns for PowerBI visualization 

In [5]:
MetricCalculation.pivot_totals_to_rows('../results/first_hospitals_before_after_reduction.csv', '../results/first_hospitals_before_after_reduction_pivoted.csv')

Transformed data saved to ../results/first_hospitals_before_after_reduction_pivoted.csv


,CTUID,Neighbourhood,Before_After_Difference,Value
7,5350001.00,South Riverdale,travel_time_to_first_dest_before,73.0
590,5350001.00,South Riverdale,travel_time_to_first_dest_after,73.0
1173,5350001.00,South Riverdale,travel_time_to_first_dest_reduction,0.0
8,5350002.00,St Lawrence-East Bayfront-The Islands,travel_time_to_first_dest_before,100.0
591,5350002.00,St Lawrence-East Bayfront-The Islands,travel_time_to_first_dest_after,100.0
...,...,...,...,...
870,5350378.27,Morningside Heights,travel_time_to_first_dest_after,84.0
1453,5350378.27,Morningside Heights,travel_time_to_first_dest_reduction,0.0
288,5350378.28,Morningside Heights,travel_time_to_first_dest_before,40.0
871,5350378.28,Morningside Heights,travel_time_to_first_dest_after,40.0


In [6]:
def group_to_ids_by_ctuid(input_path, output_path=None, from_id_col='from_id', to_id_col='to_id'):
    """
    Groups all `to_id` values for each Census Tract (`from_id` or `CTUID`) into a list.

    Args:
        input_path (str): Path to the input CSV file containing the data.
        output_path (str, optional): Path to save the grouped result as a CSV. If None, does not save.
        from_id_col (str): Column name representing the Census Tract ID (default is 'from_id').
        to_id_col (str): Column name representing the destination IDs (default is 'to_id').

    Returns:
        pd.DataFrame: A DataFrame with `from_id` and the grouped `to_id` list.
    """
    # Read the input data
    df = pd.read_csv(input_path)

    # Ensure the required columns are present
    if from_id_col not in df.columns or to_id_col not in df.columns:
        raise ValueError(f"Columns '{from_id_col}' and '{to_id_col}' must exist in the input data.")

    # Group by `from_id` and aggregate `to_id` into lists
    grouped_df = df.groupby(from_id_col)[to_id_col].apply(list).reset_index()

    # Rename the aggregated column for clarity
    grouped_df = grouped_df.rename(columns={to_id_col: f'{to_id_col}_list'})

    # Save the result to a CSV if output_path is provided
    if output_path:
        grouped_df.to_csv(output_path, index=False)
        print(f"Grouped data saved to {output_path}")

    return grouped_df

In [ ]:
MetricCalculation.group_to_ids_by_ctuid(input_path = "../results/TTM_CT_hospitals_only_before.csv",
output_path = "../results/group.csv")

Grouped data saved to ../results/group.csv


,from_id,to_id_list
0,5350001.00,"[54248, 487172, 504810, 515959, 536800, 773891..."
1,5350002.00,"[54248, 487172, 504810, 515959, 536800, 773891..."
2,5350003.00,"[54248, 487172, 504810, 515959, 536800, 773891..."
3,5350004.00,"[54248, 487172, 504810, 515959, 536800, 773891..."
4,5350005.00,"[54248, 487172, 504810, 515959, 536800, 773891..."
...,...,...
578,5350378.24,"[54248, 487172, 504810, 515959, 536800, 773891..."
579,5350378.25,"[54248, 487172, 504810, 515959, 536800, 773891..."
580,5350378.26,"[54248, 487172, 504810, 515959, 536800, 773891..."
581,5350378.27,"[54248, 487172, 504810, 515959, 536800, 773891..."


### Job Access Calculation

In [ ]:
import pandas as pd

def sum_jobs_by_from_id(grouped_to_ids_df, job_data_df, output_path=None):
    """
    Sums the jobs for each type across all `census_id` values associated with each `from_id`.

    Args:
        grouped_to_ids_df (pd.DataFrame): DataFrame containing `from_id` and lists of `to_id` values.
        job_data_df (pd.DataFrame): DataFrame containing `census_id` and job types.
        output_path (str, optional): Path to save the result as a CSV. If None, does not save.

    Returns:
        pd.DataFrame: A DataFrame with `from_id` and a list of total jobs for each type.
    """
    # Dynamically identify the ID column and job columns
    id_column = [col for col in job_data_df.columns if 'ID' in col.upper()]
    if len(id_column) != 1:
        raise ValueError("The job data must have exactly one column with 'ID' in its name.")
    id_column = id_column[0]

    job_columns = [col for col in job_data_df.columns if col != id_column]

    # Ensure the grouped DataFrame has the required columns
    if 'from_id' not in grouped_to_ids_df.columns or 'to_id_list' not in grouped_to_ids_df.columns:
        raise ValueError("Grouped data must contain 'from_id' and 'to_id_list' columns.")

    # Initialize a list to store results
    results = []

    # Iterate over each row in the grouped DataFrame
    for _, row in grouped_to_ids_df.iterrows():
        from_id = row['from_id']
        to_id_list = row['to_id_list']

        # Filter the job data for the relevant `census_id` values
        relevant_jobs = job_data_df[job_data_df[id_column].isin(to_id_list)]
        # 
        # Sum jobs across all types
        total_jobs = [relevant_jobs[col].sum() for col in job_columns]

        # Append the result
        results.append({'from_id': from_id, 'total_jobs': total_jobs})

    # Convert results to a DataFrame
    result_df = pd.DataFrame(results)

    # Save the result to a CSV if output_path is provided
    if output_path:
        result_df.to_csv(output_path, index=False)
        print(f"Summed job data saved to {output_path}")

    return result_df


In [19]:
# Extract the CTUIDs from the grouped DataFrame
grouped_ctuids = CT_within_30_grouped['from_id'].unique()

# Filter the Employment_data DataFrame to find rows where CTUID is not in the grouped CTUIDs
missing_ctuids = Employment_data[~Employment_data['CTUID'].isin(grouped_ctuids)]

# Display the rows with missing CTUIDs
display(missing_ctuids)

,CTUID,Manufacturing and Warehousing,Retail,Service,Office,Institutional,Community and Entertainment,Grand Total
0,5350001.00,982.0,661.0,1954.0,3665.0,0.0,1044.0,8306.0
1,5350002.00,37.0,0.0,174.0,1074.0,153.0,554.0,1992.0
219,5350186.00,9.0,376.0,4.0,15.0,25.0,222.0,651.0
283,5350236.02,345.0,128.0,239.0,880.0,277.0,339.0,2208.0
296,5350244.01,0.0,363.0,98.0,64.0,171.0,45.0,741.0
300,5350247.01,11389.0,1892.0,5933.0,13812.0,561.0,2706.0,36293.0
324,5350264.00,0.0,0.0,185.0,22.0,354.0,290.0,851.0
326,5350266.00,1.0,397.0,101.0,93.0,414.0,503.0,1509.0
327,5350267.00,0.0,19.0,6.0,0.0,163.0,52.0,240.0
372,5350296.00,4124.0,0.0,0.0,753.0,1594.0,299.0,6770.0


In [20]:
travel_time_matrix_path = "../results/TTM_CT_CT_before.csv"
thresholds = [30]

Job_Metric = MetricCalculation(travel_time_matrix_path)
CT_within_30 = Job_Metric.filter_destinations(threshold = thresholds)
CT_within_30_grouped = MetricCalculation.group_to_ids_by_ctuid(CT_within_30).to_csv("../results/CT_within_30_grouped.csv", index=False)

In [17]:
CT_within_30_grouped

,from_id,to_id_list
0,5350003.00,"[5350049.0, 5350048.0, 5350004.0, 5350005.0, 5..."
1,5350004.00,"[5350006.0, 5350047.04, 5350048.0, 5350007.01,..."
2,5350005.00,"[5350047.02, 5350006.0, 5350007.01, 5350007.02..."
3,5350006.00,"[5350007.01, 5350005.0, 5350004.0, 5350047.02,..."
4,5350007.01,"[5350006.0, 5350005.0, 5350004.0, 5350047.02, ..."
...,...,...
566,5350378.23,"[5350377.03, 5350378.24, 5350368.01, 5350378.0..."
567,5350378.24,"[5350378.08, 5350378.23, 5350378.06]"
568,5350378.25,[5350378.26]
569,5350378.26,"[5350378.25, 5350378.06, 5350378.17]"


In [18]:
Employment_data = pd.read_csv("../../../draft/Employment_data.csv")
Employment_data

,CTUID,Manufacturing and Warehousing,Retail,Service,Office,Institutional,Community and Entertainment,Grand Total
0,5350001.00,982.0,661.0,1954.0,3665.0,0.0,1044.0,8306.0
1,5350002.00,37.0,0.0,174.0,1074.0,153.0,554.0,1992.0
2,5350003.00,0.0,0.0,40.0,1.0,0.0,463.0,504.0
3,5350004.00,0.0,128.0,67.0,97.0,128.0,120.0,540.0
4,5350005.00,9.0,331.0,276.0,687.0,380.0,79.0,1762.0
...,...,...,...,...,...,...,...,...
576,5350378.27,1711.0,785.0,444.0,1846.0,190.0,318.0,5294.0
577,5350378.28,0.0,6.0,5.0,9.0,108.0,36.0,164.0
578,5350802.01,0.0,321.0,268.0,602.0,374.0,17.0,1582.0
579,5350802.02,2.0,6.0,9.0,0.0,53.0,7.0,77.0


In [16]:
sum_jobs_by_from_id(CT_within_30_grouped,Employment_data).to_csv("../results/1_data.csv", index=False)